<a href="https://colab.research.google.com/github/0x31nose/Deep_Learning_Tutorial/blob/master/CreatingNeuralNetowork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ニューラルネットワークの作成
1. データセットの準備と整形
2. ニューラルネットワークの定義
3. 学習と制度の検証

### あやめのデータセットの読み込み

In [1]:
from sklearn.datasets import load_iris
iris = load_iris()

In [2]:
for idx, item in enumerate(zip(iris.data, iris.target)):
  if idx == 5:
    break
  print('data', item[0], 'target:', item[1])

data [5.1 3.5 1.4 0.2] target: 0
data [4.9 3.  1.4 0.2] target: 0
data [4.7 3.2 1.3 0.2] target: 0
data [4.6 3.1 1.5 0.2] target: 0
data [5.  3.6 1.4 0.2] target: 0


In [13]:
from sklearn.model_selection import train_test_split
print("length of iris.data:", len(iris.data)) # iris.dataのデータ数
print("length of iris.target:", len(iris.target)) # iris.targetのデータ数

# iris.dataとiris.targetに含まれるデータをシャッフルして分割
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target)
print("length of X_train:", len(X_train))
print("length of y_train:", len(y_train))
print("length of X_test:", len(X_test))
print("length of y_test:", len(y_test))

length of iris.data: 150
length of iris.target: 150
length of X_train: 112
length of y_train: 112
length of X_test: 38
length of y_test: 38


In [14]:
for idx, item in enumerate(zip(X_train, y_train)):
  if idx == 5:
    break
  print('data', item[0], 'target:', item[1])

data [6.5 2.8 4.6 1.5] target: 1
data [4.8 3.  1.4 0.3] target: 0
data [7.3 2.9 6.3 1.8] target: 2
data [7.7 3.  6.1 2.3] target: 2
data [6.8 3.2 5.9 2.3] target: 2


In [15]:
import torch

X_train = torch.from_numpy(X_train).float()
y_train = torch.tensor([[float(x)] for x in y_train])
X_test = torch.from_numpy(X_test).float()
y_test = torch.tensor([[float(x)] for x in y_test])

## ニューラルネットワークの作成
- ある層のノードが次の層の全てのノードと接続されるようなニューラルネットワークを作成する。
- PyTorchではLinearクラス（torch.nn.Linearクラス）を使うことで簡潔に記述できる。
- 以下では、入力データ→入力層→活性化関数→隠れ層→（出力層→）出力という流れを書いてみる

In [21]:
# PyTorchが提供するtorch.nnモジュールをインポートする
from torch import nn

# 入力層、隠れ層、出力の数を変数に代入する
INPUT_FEATURES = 4
HIDDEN = 5
OUTPUT_FEATURES = 1

# nn.Moduleクラス（torch.nn.Moduleクラス）を基底クラスとする
# そうすることで、Pytorchが提供するニューラルネットワークのすべての機能を継承する
# その後に、__init__とforwardの２つのメソッドを定義する
class Net(nn.Module):
  #　基底クラスの__init__メソッドを呼び出して、初期化を行っている
  def __init__(self):
    super().__init__()
    # 入力層を表すインスタンス（入力層→隠れ層）
    self.fc1 = nn.Linear(INPUT_FEATURES, HIDDEN)
    # 隠れ層を表すインスタンス(隠れ層→出力層)
    self.fc2 = nn.Linear(HIDDEN, OUTPUT_FEATURES)

  # ニューラルネットがどのように計算を連ねていくかを定める
  # 入力xを受け取り、self.fc1で処理し、
  # その結果をsigmoid関数（活性化関数）に通して、
  # self.fc2メソッドで処理して、その結果を戻り値としている
  def forward(self, x):
    x = self.fc1(x)
    x = torch.sigmoid(x)
    x = self.fc2(x)
    return x

In [32]:
# とりあえず訓練してないけど使ってみるテスト
net = Net() #

outputs = net(X_train[0:3]) # 訓練データの先頭から３個の要素を入力
print(outputs)
for idx in range(3):
  print('output', outputs[idx], ', label:', y_train[idx])

tensor([[-0.2203],
        [-0.1635],
        [-0.2107]], grad_fn=<AddmmBackward>)
output tensor([-0.2203], grad_fn=<SelectBackward>) , label: tensor([1.])
output tensor([-0.1635], grad_fn=<SelectBackward>) , label: tensor([0.])
output tensor([-0.2107], grad_fn=<SelectBackward>) , label: tensor([2.])


### 学習
#### 学習の概要
- ニューラルネットワークにデータを入力
- 得られる計算結果（推測結果）と正解ラベルを比較
- ニューラルネットワークが持つ重みやバイアスを更新する
- 計算結果と正解ラベルとの誤差を比べる
- 最適化と呼ばれる処置を行う

#### 学習の手順
1. ニューラルネットワークにX_trainに格納したデータを入力する
2. 損失関数を用いて計算結果と正解ラベルとの誤差を計算する
3. 誤差逆伝播や最適化処理によって重みやバイアスを更新
4. １〜３を繰り返す

In [39]:
net = Net()

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.003)

EPOCHS = 3000
for epoch in range(EPOCHS):
  optimizer.zero_grad()
  outputs = net(X_train)
  loss = criterion(outputs, y_train)
  loss.backward()
  optimizer.step()

  if epoch % 100 == 99:
    print(f'epoch: {epoch+1:4}, loss: {loss.data}')

print('training finished')

epoch:  100, loss: 0.5484613180160522
epoch:  200, loss: 0.5020634531974792
epoch:  300, loss: 0.45815515518188477
epoch:  400, loss: 0.4139237403869629
epoch:  500, loss: 0.36891841888427734
epoch:  600, loss: 0.32376593351364136
epoch:  700, loss: 0.28034043312072754
epoch:  800, loss: 0.24059738218784332
epoch:  900, loss: 0.20580913126468658
epoch: 1000, loss: 0.1764279305934906
epoch: 1100, loss: 0.15228192508220673
epoch: 1200, loss: 0.13283689320087433
epoch: 1300, loss: 0.11740414053201675
epoch: 1400, loss: 0.1052742525935173
epoch: 1500, loss: 0.09579122811555862
epoch: 1600, loss: 0.08838657289743423
epoch: 1700, loss: 0.08258896321058273
epoch: 1800, loss: 0.07802040129899979
epoch: 1900, loss: 0.07438546419143677
epoch: 2000, loss: 0.07145781815052032
epoch: 2100, loss: 0.06906678527593613
epoch: 2200, loss: 0.06708456575870514
epoch: 2300, loss: 0.06541653722524643
epoch: 2400, loss: 0.06399232894182205
epoch: 2500, loss: 0.06275959312915802
epoch: 2600, loss: 0.061679176

In [40]:
# 学習前に行った結果よりかは近しい値が出力されている。
for idx, item in enumerate(zip(outputs, y_train)):
  if idx == 5:
    break
  print(item[0].data, '<-->', item[1])

tensor([1.2925]) <--> tensor([1.])
tensor([0.0662]) <--> tensor([0.])
tensor([1.8220]) <--> tensor([2.])
tensor([1.8194]) <--> tensor([2.])
tensor([1.8496]) <--> tensor([2.])


In [41]:
predict = (outputs + 0.5).int()
for idx, item in enumerate(zip(predict, y_train)):
  if idx == 5:
    break
  print('output:', item[0], ', label:', item[1])

output: tensor([1], dtype=torch.int32) , label: tensor([1.])
output: tensor([0], dtype=torch.int32) , label: tensor([0.])
output: tensor([2], dtype=torch.int32) , label: tensor([2.])
output: tensor([2], dtype=torch.int32) , label: tensor([2.])
output: tensor([2], dtype=torch.int32) , label: tensor([2.])


In [51]:
compare = predict == y_train
print(compare[0:5])
print("Accuracy:", int(compare.sum())/112)

tensor([[True],
        [True],
        [True],
        [True],
        [True]])
Accuracy: 0.9821428571428571


In [55]:
outputs = net(X_test)

predict = (outputs + 0.5).int()
compare = predict == y_test

print(f'correct: {compare.sum()} / {len(predict)}')
for value, label in zip(predict, y_test):
  print('predicted:', iris.target_names[value.item()], '<--->',
        'label:', iris.target_names[int(label.item())])

correct: 38 / 38
predicted: setosa <---> label: setosa
predicted: setosa <---> label: setosa
predicted: virginica <---> label: virginica
predicted: versicolor <---> label: versicolor
predicted: setosa <---> label: setosa
predicted: setosa <---> label: setosa
predicted: versicolor <---> label: versicolor
predicted: versicolor <---> label: versicolor
predicted: setosa <---> label: setosa
predicted: virginica <---> label: virginica
predicted: setosa <---> label: setosa
predicted: versicolor <---> label: versicolor
predicted: setosa <---> label: setosa
predicted: setosa <---> label: setosa
predicted: setosa <---> label: setosa
predicted: versicolor <---> label: versicolor
predicted: setosa <---> label: setosa
predicted: virginica <---> label: virginica
predicted: virginica <---> label: virginica
predicted: virginica <---> label: virginica
predicted: virginica <---> label: virginica
predicted: virginica <---> label: virginica
predicted: virginica <---> label: virginica
predicted: versicolor

###  重みとバイアス